In [89]:
print("ok!")

ok!


In [90]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [91]:
PINECONE_API_KEY = "02d048ed-ed5e-4d23-ab10-8df2e59b9f2a"
PINECONE_API_ENV = "gcp-starter"

In [92]:
#Extract data from the pdf 
def load_pdf(data):

    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader
                             )

    documents = loader.load()

    return documents


In [93]:
extracted_data = load_pdf("T:\Destktop\m_chatbot\Data")

In [94]:
extracted_data

[Document(page_content='Global Orange Dev elopment,  LLC \nFDD Dated April 30, 2021   FRANCHISE DISCLOSURE DOCUMENT  \n \nGLOBAL ORANGE DEVELOPMENT, LLC \na Michigan limited liability company  \n2501 Coolidge Road, #302  \nEast Lansing, Michigan 48823  \nPhone: (517) 913 -1983  \nEmail: franchiseinfo@biggby.com    \nWebsite: www.biggby.com   \n \n \nThe BIGGBY® COFFEE  franchise is a community coff ee shop with offerings that include espresso-\nbased beverages, coffee, tea, energy beverages, other beverages , sandwiches , baked goods, and \nother food, whole bean  coffee  including farm direct option s, merchandise and coffee accessories.  \n \nThe total investment necessary to begin operation of a BIGGBY® COFFEE  franchise is from \n$202,450 to $4 18,700.  This includes  $25,750 to $38,750  that must be p aid to the franchisor or \nits affiliates.   \n This dis closure document summarizes certain provision of your franchise agreement and other \ninformation in plain English.  Read thi

In [95]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=128,chunk_overlap=5)
    text_chunks =text_splitter.split_documents(extracted_data)
    return text_chunks


In [96]:
text_chunks = text_split(extracted_data)
print("length of my chunks",len(text_chunks))

length of my chunks 9356


In [97]:
text_chunks

[Document(page_content='Global Orange Dev elopment,  LLC \nFDD Dated April 30, 2021   FRANCHISE DISCLOSURE DOCUMENT  \n \nGLOBAL ORANGE DEVELOPMENT, LLC', metadata={'source': 'T:\\Destktop\\m_chatbot\\Data\\1.pdf', 'page': 0}),
 Document(page_content='a Michigan limited liability company  \n2501 Coolidge Road, #302  \nEast Lansing, Michigan 48823  \nPhone: (517) 913 -1983', metadata={'source': 'T:\\Destktop\\m_chatbot\\Data\\1.pdf', 'page': 0}),
 Document(page_content='Email: franchiseinfo@biggby.com    \nWebsite: www.biggby.com', metadata={'source': 'T:\\Destktop\\m_chatbot\\Data\\1.pdf', 'page': 0}),
 Document(page_content='The BIGGBY® COFFEE  franchise is a community coff ee shop with offerings that include espresso-', metadata={'source': 'T:\\Destktop\\m_chatbot\\Data\\1.pdf', 'page': 0}),
 Document(page_content='based beverages, coffee, tea, energy beverages, other beverages , sandwiches , baked goods, and', metadata={'source': 'T:\\Destktop\\m_chatbot\\Data\\1.pdf', 'page': 0}),


In [98]:
#Download embedding model 
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name ="BAAI/bge-base-en-v1.5")
    return embeddings

In [99]:
embedding = download_hugging_face_embedding()

In [100]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='BAAI/bge-base-en-v1.5', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [101]:
query_result = embedding.embed_query("hello world")
print("length",len(query_result))

length 768


In [102]:
from langchain.vectorstores import Pinecone as PineconeVectorStore

In [103]:
pinecone.init(api_key = PINECONE_API_KEY,
              environment = PINECONE_API_ENV)
index_name = "chatbot"

docsearch =PineconeVectorStore.from_texts([t.page_content for t in text_chunks],embedding=embedding,index_name = index_name)

In [104]:
docsearch = Pinecone.from_existing_index(index_name,embedding)

query = "what are the name of franchises"

docs = docsearch.similarity_search(query,k=3)
print("Result",docs)

Result [Document(page_content='FRANCHISE O WNER  \n \n \n \nBy:       \n \nIts:', metadata={}), Document(page_content='a similar business during th e term of the franchise.  There are usually other', metadata={}), Document(page_content='a similar business during th e term of the franchise.  There are usually other', metadata={})]


In [105]:
prompt_template = """
You are a Medicine Representative.You are designed to be helpful
as possible while providing only factual information.
you should be friendly,but not over chatty.
Context information is below,Given the context information and 
not prior knowledge,answer the question.
Context: {context}
Question:{question}
Only return helpful answer below and nothing else.
Helpful answer:
"""



In [106]:
PROMPT = PromptTemplate(template=prompt_template,input_variables=["context","question"])
chain_type_kwargs = {"prompt":PROMPT}

In [107]:
llm = CTransformers(model = "T:\Destktop\m_chatbot\model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens':512,
                            'temperature':0.1})

In [108]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="stuff",
    retriever = docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents = True,
    chain_type_kwargs= chain_type_kwargs
)

In [110]:
while True:
    user_input = input(f"Input Prompt:")
    result = qa({'query':user_input})
    print("Response: ",result["result"])

Response:  The estimated investment range for this item is $10,000 to $20,000.
Response:  The investment range for a BIGGBY COFFEE franchise varies depending on several factors, including location, size of the store, and the number of locations you plan to open. According to the company's website, the estimated initial investment for a single location ranges from $350,000 to $1.2 million. This includes costs such as franchise fees, construction and build-out expenses, inventory, equipment, and marketing. It is important to note that this is just an estimate and the actual cost of opening a BIGGBY COFFEE franchise may be higher or lower depending on your specific circumstances.


KeyboardInterrupt: 

In [ ]:
user_input = input(f"Input Prompt:")
result = qa({'query':user_input})
print("Response: ",result["result"])

PineconeProtocolError: Failed to connect; did you specify the correct index name?